In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from shapely import wkt
import re

os.chdir('../') # set working directory to root of repo

/home/xuyuan/.local/lib/python3.8/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_5454/3318037310.py:4: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly 

now we extract the number of transactions within the given 1km geometry to map with the community level data.

## the below codes are for merging the 1km data to the district level data

you do not need to execute them because I have already merged it

In [99]:
data = pd.read_csv('cleaned_1km.csv')
df = pd.read_csv('uncleaned_district.csv')
data.rename(columns={'prft':'district_income', 'num': 'district_num', 'price': 'district_price'}, inplace=True)
data.drop(columns = 'index', inplace=True)

# Create GeoDataFrames
df_copy = gpd.GeoDataFrame(df.copy(), geometry=df['geometry'].apply(wkt.loads))
data_copy = gpd.GeoDataFrame(data.copy(), geometry=data['geometry'].apply(wkt.loads))
des = gpd.GeoDataFrame(data.drop_duplicates(subset=['geometry'], keep='first').copy(), 
                       geometry=data.drop_duplicates(subset=['geometry'], keep='first')['geometry'].apply(wkt.loads))


In [100]:
data.columns

Index(['Unnamed: 0', 'year', 'top', 'left', 'id', 'right', 'bottom', 'ngtm',
       'living', 'flnum', 'wtpp', 'tihu', 'pripe', 'mall', 'ldtm', 'numbu',
       'diff', 'west', 'kind', 'hotel', 'muse', 'super', 'nego', 'jd',
       'grrate', 'prim', 'broke', 'dfrat', 'district_income', 'flrate',
       'reside', 'district_num', 'wttm', 'toilet', 'ktv', 'bedroo', 'peri',
       'park', 'lttm', 'mid', 'district_price', 'kitche', 'old', 'area',
       'fllvl', 'age', 'sub', 'lianj', 'light', 'pop', 'pm25', 'geometry',
       'city', 'id_unique', 'point_value', 'grppr', 'grpnu', 'grpic'],
      dtype='object')

In [101]:
df.columns

Index(['building_type', 'village', 'district', 'floor_level', 'new_lng',
       'new_lat', 'year', 'floor_ratio', 'green_ratio', 'nego_times',
       'lead_times', 'total_building', 'total_resident', 'watching_people',
       'watched_times', 'striker_price', 'striker_price_pers', 'end_price',
       'end_price_pers', 'area', 'nego_period', 'bedroom', 'living_room',
       'kitchen', 'toilet', 'total_floor_number', 'elevator_ratio',
       'house_age', 'income', 'number', 'super', 'sub', 'hotel', 'kind',
       'prim', 'mid', 'shop_mall', 'west_food', 'park', 'museum', 'ktv',
       'jiadian', 'old', 'other', 'other_5', 'lianjia', 'lianjia_5', 'beke',
       'beke_5', 'geometry', 'light', 'pop', 'pm25', 'region', 'id',
       'business_area'],
      dtype='object')

In [102]:
df_copy = df_copy.set_crs('epsg:4326')
data_copy = data_copy.set_crs('epsg:4326')
des = des.set_crs('epsg:4326')

In [103]:
data_copy['district_price'] = data_copy['district_price'] / data_copy['area']
# this would guarantee that the result is the average price per square meter

In [104]:
if 'index_left' in df_copy.columns:
    df_copy.drop(columns=['index_left'], inplace=True)
if 'index_right' in df_copy.columns:
    df_copy.drop(columns=['index_right'], inplace=True)

# Check and rename/drop 'index_left' and 'index_right' in des
if 'index_left' in des.columns:
    des.drop(columns=['index_left'], inplace=True)
if 'index_right' in des.columns:
    des.drop(columns=['index_right'], inplace=True)
    
joined_gdf = gpd.sjoin(df_copy, des, how="left", op='within')
joined_gdf.drop(columns = ['district_income', 'district_num', 'district_price'], inplace = True)

/home/xuyuan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [106]:
# joined_gdf['id_unique', 'year'] # id_unique is the unique identification of the data object
# and now we shall join the data to the district data

data_copy['unique_key'] = data_copy['id_unique'].astype(str) + '_' + data_copy['year'].astype(str)
joined_gdf['unique_key'] = joined_gdf['id_unique'].astype(str) + '_' + joined_gdf['year_left'].astype(str)
data_relevant = data_copy[['unique_key', 'district_income', 'district_num', 'district_price']]

joined_gdf = joined_gdf.merge(data_relevant, on='unique_key', how='left')
joined_gdf.drop(columns=['unique_key'], inplace=True)

In [107]:
df_copy['unique_key'] = df_copy['id'].astype(str) + '_' + df_copy['year'].astype(str)
joined_gdf['unique_key'] = joined_gdf['id_left'].astype(str) + '_' + joined_gdf['year_left'].astype(str)

In [110]:
joined_gdf.rename(columns = {'district_income': 'region_income', 'district_num': 'region_num', 'district_price': 'region_price'}, inplace=True)
data_relevant = joined_gdf[['unique_key', 'region_income', 'region_num', 'region_price']]

In [111]:
df_copy = df_copy.merge(data_relevant, on='unique_key', how='left')

In [113]:
df_copy.to_csv('cleaned_district_Jan.csv', index = False)

In [114]:
data = pd.read_csv('cleaned_district_Jan.csv')

In [115]:
data.sort_values(by = ['id', 'year'], inplace = True)
data['lag_lianjia'] = data.groupby('id')['lianjia_5'].shift(1)
data['lag_lianjia'] = data['lag_lianjia'].fillna(data['lianjia_5'])
data[['id', 'lianjia_5', 'lag_lianjia']]
data['entry'] = (data['lianjia_5'] > data['lag_lianjia']).astype(int)

### note that this function is wrong!!!!!!!!
# and we should just drop it right here

In [116]:
n = 3
for i in range(1, n + 1):
    data[f'post{i}'] = data.groupby('id')['entry'].shift(i).fillna(0)
for i in range(1, n + 1):
    data[f'pre{i}'] = data.groupby('id')['entry'].shift(-i).fillna(0)

In [117]:
data.to_csv('cleaned_district_Jan_2.csv', index = None)

## now we construct the RD design dataset

the RD design dataset contains the following properties:

First, we extract a list of csv files located in the lianjia_beke directory within the given working path. Then we map these files to the design of communities and extract each lianjia store with its nearest community or nearest two communities respectively. Then we conduct the RD analysis in the stata file.

In [118]:
data = pd.read_csv('cleaned_district_Jan_2.csv')

In [119]:

# Define the directory path
directory_path = "lianjia_beke"
filenames = []
# List all files and directories in the given path
for filename in os.listdir(directory_path):
    filenames.append(filename)
# data['region'].unique()

In [120]:
city_name_mapping = {
    'beijing': '北京市',
    'chengdu': '成都市',
    'chongqing': '重庆市',
    'guangzhou': '广州市',
    'hangzhou': '杭州市',
    'nanjing': '南京市',
    'shanghai': '上海市',
    'shenzhen': '深圳市',
    'tianjin': '天津市',
    'wuhan': '武汉市',
    'xian': '西安市'
}

In [121]:
def map_to_chinese_csv(row):
    year_suffix = str(row['year'])[-2:]  # Extract the last two digits of the year
    chinese_city = city_name_mapping[row['region']]  # Map to Chinese city name
    return f"{chinese_city}{year_suffix}.csv"  # Combine to form the Chinese CSV file name

In [122]:
data['chinese_csv'] = data.apply(map_to_chinese_csv, axis=1)

In [123]:
dataframes_list = []
for i in data['chinese_csv'].unique():
    dataframes_list.append(data[data['chinese_csv'] == i])

for i in range(len(dataframes_list)):
    dataframes_list[i] = gpd.GeoDataFrame(dataframes_list[i], geometry=dataframes_list[i]['geometry'].apply(wkt.loads))
    dataframes_list[i] = dataframes_list[i].set_crs('epsg:4326')

## NOTE

the code below extract the number of nearest community for each lianjia's store. It used nested for loop to achieve this and this code runs pretty long time, we can optimize it using Cpp.

For computing purpose, I suppose to use the Cpp codes in the second block

procedure to build it (you should revise the location in your computer):



```
pip install pybind11

python3-config --cflags

pybind11-config --includes

cd the/file/path/RealEstateBrokerage/

g++ -O3 -shared -std=c++11 -fPIC -I/usr/include/python3.8 -I/home/xuyuan/.local/lib/python3.8/site-packages/pybind11/include -o nearest_community_cpp.so nearest_community_cpp.cpp
```

the running time is less than 25 seconds

```
for i in range(0, len(dataframes_list)):
    communities_gdf = dataframes_list[i]
    df = pd.read_csv('lianjia_beke/' + communities_gdf['chinese_csv'].unique()[0])
    store_locations_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['gpsx'], df['gpsy']))
    # store_locations_gdf['nearest_community_index'] = -1
    
    num_nearest_communities = 2
    
    nearest_community_indices_list = []
    
    for store_index, store_row in store_locations_gdf.iterrows():
        store_location = store_row['geometry']
        
        # Calculate distances to all communities and store them in a Series
        distances = communities_gdf.geometry.apply(lambda x: store_location.distance(x))
        
        # Sort the distances and select the indices of the nearest communities
        nearest_community_indices = distances.argsort()[:num_nearest_communities].tolist()
        
        # Append the list of nearest community indices to the list
        nearest_community_indices_list.append(nearest_community_indices)
        
    # Assign the list to the 'nearest_community_indices' column
    store_locations_gdf['nearest_community_indices'] = nearest_community_indices_list
    
    store_locations_gdf.to_csv('nearest_community/' + communities_gdf['chinese_csv'].unique()[0], index = False)
```

In [124]:
import nearest_community_cpp

directory_path = "nearest_community"
if not os.path.exists(directory_path):
    # If it doesn't exist, create the directory
    os.makedirs(directory_path)

for i in range(0, len(dataframes_list)):
    communities_gdf = dataframes_list[i]
    community_locations_cpp = [list(point.coords[0]) for point in communities_gdf['geometry']]
    df = pd.read_csv('lianjia_beke/' + communities_gdf['chinese_csv'].unique()[0])
    store_locations_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['gpsx'], df['gpsy']))
    store_locations_cpp = [list(point.coords[0]) for point in store_locations_gdf['geometry']]
    
    num_nearest_communities = 2 # you may change this number whatever you want
    nearest_community_indices_tuple = \
        nearest_community_cpp.find_nearest_communities(store_locations_cpp, community_locations_cpp, num_nearest_communities)
    
    nearest_community_indices_list, nearest_community_distances_list = nearest_community_indices_tuple

    # Assign the list to the 'nearest_community_indices' column
    store_locations_gdf['nearest_community_indices'] = nearest_community_indices_list
    store_locations_gdf['nearest_community_distances'] = nearest_community_distances_list
    
    store_locations_gdf.to_csv('nearest_community/' + communities_gdf['chinese_csv'].unique()[0], index = False)

now we merge it back to our dataframe to get the original dataframe. Note that the order gonna be:

1. The index of the nearest community to the store location.
2. The index of the second nearest community to the store location.
3. If there are more communities to consider (i.e., num_nearest_communities is greater than 2), the indices of the subsequent nearest communities will follow.

In [125]:
communities_gdf = dataframes_list[0]
df = pd.read_csv('nearest_community/' + communities_gdf['chinese_csv'].unique()[0])
df.head(5)

,name,type,gpsx,gpsy,lianjia2,geometry,nearest_community_indices,nearest_community_distances
0,链家(潘家园店),生活服务;中介机构;中介机构|购物服务;购物相关场所;购物相关场所,116.456334,39.875065,链家(潘家园店),POINT (116.456333588003 39.8750648736051),"[3016, 870]","[0.017490703063189963, 0.05503946806844407]"
1,链家(团结湖店),生活服务;中介机构;中介机构|购物服务;购物相关场所;购物相关场所,116.461314,39.927974,链家(团结湖店),POINT (116.461313947982 39.9279739298663),"[1591, 1578]","[0.19038161099272874, 0.23991664301711862]"
2,链家(团结湖路),生活服务;中介机构;中介机构|购物服务;购物相关场所;购物相关场所,116.461018,39.926613,链家(团结湖路),POINT (116.461018145355 39.9266132293729),"[1591, 1594]","[0.08929879133855328, 0.14133020776835956]"
3,链家(金星园店),生活服务;中介机构;中介机构|购物服务;购物相关场所;购物相关场所,116.449045,39.966118,链家(金星园店),POINT (116.449045305641 39.9661181413891),"[4814, 2183]","[0.023880360645829844, 0.13137326907335756]"
4,链家(花家地西里店),生活服务;中介机构;中介机构|购物服务;购物相关场所;购物相关场所,116.451689,39.987442,链家(花家地西里店),POINT (116.451689294448 39.987442214481696),"[519, 4217]","[0.22282636588868104, 0.26696901573145504]"


In [126]:
revised_dataframe = pd.DataFrame()
for i in range(0, len(dataframes_list)):
    communities_gdf = pd.DataFrame(dataframes_list[i])
    
    df = pd.read_csv('nearest_community/' + communities_gdf['chinese_csv'].unique()[0])
    
    communities_gdf['nearest_index_1'] = 0
    communities_gdf['nearest_index_2'] = 0
    
    for _, row in df.iterrows():
        nearest_indices_str = row['nearest_community_indices']
        nearest_indices = [int(num) for num in re.findall(r'\d+', nearest_indices_str)]
        
        # Update communities_gdf for the first nearest community
        if len(nearest_indices) > 0 and nearest_indices[0] in communities_gdf.index:
            communities_gdf.loc[nearest_indices[0], 'nearest_index_1'] = 1
            communities_gdf.loc[nearest_indices[0], 'nearest_index_2'] = 1
        
        # Update communities_gdf for the second nearest community
        if len(nearest_indices) > 1 and nearest_indices[1] in communities_gdf.index:
            communities_gdf.loc[nearest_indices[1], 'nearest_index_2'] = 1
    
    revised_dataframe = pd.concat([communities_gdf, revised_dataframe])

In [127]:
revised_dataframe.to_csv('cleaned_district_Jan_3.csv', index = False)

## Calculate each community's distance to each store

the cpp file calculates each community's smallest distance to the stores

```
g++ -O3 -shared -std=c++11 -fPIC -I/usr/include/python3.8 -I/home/xuyuan/.local/lib/python3.8/site-packages/pybind11/include -o nearest_stores.so nearest_stores.cpp
```

In [128]:
data = pd.read_csv('cleaned_district_Jan_3.csv')

In [129]:
dataframes_list = []
for i in data['chinese_csv'].unique():
    dataframes_list.append(data[data['chinese_csv'] == i])

for i in range(len(dataframes_list)):
    dataframes_list[i] = gpd.GeoDataFrame(dataframes_list[i], geometry=dataframes_list[i]['geometry'].apply(wkt.loads))
    dataframes_list[i] = dataframes_list[i].set_crs('epsg:4326')

In [130]:
import nearest_stores

result_dataframe = pd.DataFrame()

for i in range(0, len(dataframes_list)):
    communities_gdf = dataframes_list[i]
    community_locations_cpp = [list(point.coords[0]) for point in communities_gdf['geometry']]
    df = pd.read_csv('lianjia_beke/' + communities_gdf['chinese_csv'].unique()[0])
    store_locations_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['gpsx'], df['gpsy']))
    store_locations_cpp = [list(point.coords[0]) for point in store_locations_gdf['geometry']]
    
    num_nearest_communities = 1 # you may change this number whatever you want
    nearest_store_indices_tuple = \
        nearest_stores.find_nearest_stores_to_communities(store_locations_cpp, community_locations_cpp, num_nearest_communities)
    
    nearest_store_indices, nearest_store_distances = nearest_store_indices_tuple

    # Assign the list to the 'nearest_community_indices' column
    communities_gdf['nearest_store_indices'] = nearest_store_indices
    communities_gdf['nearest_store_distances'] = nearest_store_distances
    
    communities_gdf = pd.DataFrame(communities_gdf)
    
    result_dataframe = pd.concat([communities_gdf, result_dataframe])
    # store_locations_gdf.to_csv('nearest_community/' + communities_gdf['chinese_csv'].unique()[0], index = False)

since we are working with nearest distance, we can directly convert it to double

In [131]:
result_dataframe['nearest_store_distances'] = result_dataframe['nearest_store_distances'].apply(lambda x: x[0])

In [132]:
result_dataframe.to_csv('cleaned_district_Jan_4.csv', index = False)

## removing temperary files

the following command remove all temperary files generated during our code

In [133]:
def delete_file(filename):
    try:
        os.remove(filename)
        print(f"{filename} has been deleted.")
    except FileNotFoundError:
        print(f"File: {filename} not found.")
    except PermissionError:
        print(f"Permission denied.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Replace 'my_file.txt' with the name of the file you want to delete
delete_file('cleaned_district_Jan.csv')
delete_file('cleaned_district_Jan_2.csv')
delete_file('cleaned_district_Jan_3.csv')

cleaned_district_Jan.csv has been deleted.
cleaned_district_Jan_2.csv has been deleted.
cleaned_district_Jan_3.csv has been deleted.


In [2]:
data = pd.read_csv('cleaned_district_Jan_4.csv')

## now you need to refer to the classifying dataframe

after you have done that dataframe, you would obtain a cleaned_district_Jan_5.csv

In [22]:
data = pd.read_csv('cleaned_district_Jan_5.csv')

In [23]:
data.sort_values(by = ['id', 'year'], inplace = True)
data.reset_index(drop = True, inplace = True)
data['lag_lianjia'] = data.groupby('id')['lianjia_420'].shift(1)
data['lag_lianjia'] = data['lag_lianjia'].fillna(data['lianjia_420'])
data[['id', 'lianjia_420', 'lag_lianjia']]
data['potential_entry'] = (data['lianjia_420'] > data['lag_lianjia']).astype(int)

# Initialize the 'entry' column with zeros (no entry)
data['entry'] = 0

# For each group (each 'id'), find the first occurrence where 'potential_entry' is 1 and mark it as the entry
for id_val in data['id'].unique():
    first_entry_index = data[(data['id'] == id_val) & (data['potential_entry'] == 1)].index.min()
    if pd.notna(first_entry_index):
        data.at[first_entry_index, 'entry'] = 1
data.drop('potential_entry', axis=1, inplace=True)

In [24]:
n = 3
for i in range(1, n + 1):
    data[f'post{i}'] = data.groupby('id')['entry'].shift(i).fillna(0)
for i in range(1, n + 1):
    data[f'pre{i}'] = data.groupby('id')['entry'].shift(-i).fillna(0)

In [25]:
data['lag_lianjia1'] = data.groupby('id')['lianjia_5'].shift(1)
data['lag_lianjia1'] = data['lag_lianjia1'].fillna(data['lianjia_5'])
data[['id', 'lianjia_5', 'lag_lianjia']]


data['potential_entry'] = (data['lianjia_5'] > data['lag_lianjia1']).astype(int)

# Initialize the 'entry' column with zeros (no entry)
data['entry1'] = 0

# For each group (each 'id'), find the first occurrence where 'potential_entry' is 1 and mark it as the entry
for id_val in data['id'].unique():
    first_entry_index = data[(data['id'] == id_val) & (data['potential_entry'] == 1)].index.min()
    if pd.notna(first_entry_index):
        data.at[first_entry_index, 'entry1'] = 1
data.drop('potential_entry', axis=1, inplace=True)

n = 3
for i in range(1, n + 1):
    data[f'postr{i}'] = data.groupby('id')['entry1'].shift(i).fillna(0)
for i in range(1, n + 1):
    data[f'prer{i}'] = data.groupby('id')['entry1'].shift(-i).fillna(0)

I think the idea should be if the lianjia has been in the market before our study happens, then we should not give it a effect of entry. 

We just observe the entry effect, and we do not care about the already built in effect.

Besides, the data is so confused, and there are multiple periods before and after the entry, how to deal with them. Since this is a fuzzy DID like result, that the reference groups is hard to identify.

In [26]:

data.to_csv('cleaned_district_Jan_6.csv', index = None)